In [1]:
param_x_api_key = '2BeBXg68o3pSd6GQvdyS5erm1aSyJUdKxZcgwsrEukJHa1mYjko5'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '{"getAutoCarePartApplications":{"brandCode":"' + input_.loc[a, 'Brand_Id'] + '","partNumber":"' + input_.loc[a, 'Part_Number']  + '","includeResults":false,"includeMatchingBaseVehicles":false,"perPage":100}}'

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                vehicle_page = math.ceil(json_data['total'] / 100)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_PNBI': input_.loc[a, 'JOIN_PNBI'],
                                         'Vehicle_Page': vehicle_page}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Brand_Id': input_.loc[a, 'Brand_Id'],
                                     'JOIN_PNBI': input_.loc[a, 'JOIN_PNBI']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'JOIN_PNBI'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNBI'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle_page-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand_Id'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_page_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1499

[ok] - 1003-1027;DKHD
[尝试次数：4] - [剩余数量：1492] - [当前时间：11:04:05]

[ok] - 1003-1029;DKHD
[尝试次数：1] - [剩余数量：1491] - [当前时间：11:04:06]

[ok] - 1003-1022;DKHD
[尝试次数：6] - [剩余数量：1490] - [当前时间：11:04:06]

[ok] - 1003-1026;DKHD
[尝试次数：6] - [剩余数量：1489] - [当前时间：11:04:07]

[ok] - 1003-1028;DKHD
[尝试次数：5] - [剩余数量：1488] - [当前时间：11:04:07]

[ok] - 1003-1024;DKHD
[尝试次数：11] - [剩余数量：1487] - [当前时间：11:04:07]

[ok] - 1003-1037;DKHD
[尝试次数：2] - [剩余数量：1486] - [当前时间：11:04:08]

[ok] - 1003-1039;DKHD
[尝试次数：4] - [剩余数量：1485] - [当前时间：11:04:09]

[ok] - 1003-1038;DKHD
[尝试次数：4] - [剩余数量：1484] - [当前时间：11:04:09]

[ok] - 1003-1040;DKHD
[尝试次数：4] - [剩余数量：1483] - [当前时间：11:04:10]

[ok] - 1003-1031;DKHD
[尝试次数：7] - [剩余数量：1482] - [当前时间：11:04:10]

[ok] - 1003-1032;DKHD
[尝试次数：6] - [剩余数量：1481] - [当前时间：11:04:10]

[ok] - 1003-1025;DKHD
[尝试次数：18] - [剩余数量：1480] - [当前时间：11:04:10]

[ok] - 1003-1042;DKHD
[尝试次数：1] - [剩余数量：1479] - [当前时间：11:04:11]

[ok] - 1003-1044;DKHD
[尝试次数：3] - [剩余数量：1478] - [当前时间：11:04:11]

[ok] - 1003-1055;DKHD
[尝试次数：

[ok] - 210-1122;DKHD
[尝试次数：1] - [剩余数量：1364] - [当前时间：11:05:49]

[ok] - 210-1129;DKHD
[尝试次数：1] - [剩余数量：1363] - [当前时间：11:05:49]

[ok] - 210-1127;DKHD
[尝试次数：2] - [剩余数量：1362] - [当前时间：11:05:49]

[ok] - 210-1130;DKHD
[尝试次数：1] - [剩余数量：1361] - [当前时间：11:05:50]

[ok] - 210-1131;DKHD
[尝试次数：2] - [剩余数量：1360] - [当前时间：11:05:50]

[ok] - 210-1133;DKHD
[尝试次数：1] - [剩余数量：1359] - [当前时间：11:05:50]

[ok] - 210-1139;DKHD
[尝试次数：1] - [剩余数量：1358] - [当前时间：11:05:51]

[ok] - 210-1134;DKHD
[尝试次数：4] - [剩余数量：1357] - [当前时间：11:05:52]

[ok] - 210-1141;DKHD
[尝试次数：2] - [剩余数量：1356] - [当前时间：11:05:53]

[ok] - 210-1140;DKHD
[尝试次数：4] - [剩余数量：1355] - [当前时间：11:05:54]

[ok] - 1003-1162;DKHD
[尝试次数：19] - [剩余数量：1354] - [当前时间：11:05:56]

[ok] - 210-1145;DKHD
[尝试次数：1] - [剩余数量：1353] - [当前时间：11:05:56]

[ok] - 210-1142;DKHD
[尝试次数：1] - [剩余数量：1352] - [当前时间：11:05:57]

[ok] - 210-1148;DKHD
[尝试次数：1] - [剩余数量：1351] - [当前时间：11:05:57]

[ok] - 210-1106;DKHD
[尝试次数：2] - [剩余数量：1350] - [当前时间：11:05:58]

[ok] - 210-1149;DKHD
[尝试次数：1] - [剩余数量：1349] - [当前时间：1

[ok] - 1003-1074;DKHD
[尝试次数：1] - [剩余数量：1234] - [当前时间：11:07:10]

[ok] - 1003-1082;DKHD
[尝试次数：1] - [剩余数量：1233] - [当前时间：11:07:11]

[ok] - 1003-1083;DKHD
[尝试次数：2] - [剩余数量：1232] - [当前时间：11:07:11]

[ok] - 1003-1077;DKHD
[尝试次数：1] - [剩余数量：1231] - [当前时间：11:07:11]

[ok] - 1003-1072;DKHD
[尝试次数：1] - [剩余数量：1230] - [当前时间：11:07:11]

[ok] - 1003-1098;DKHD
[尝试次数：1] - [剩余数量：1229] - [当前时间：11:07:12]

[ok] - 1003-1086;DKHD
[尝试次数：2] - [剩余数量：1228] - [当前时间：11:07:12]

[ok] - 1003-1087;DKHD
[尝试次数：3] - [剩余数量：1227] - [当前时间：11:07:13]

[ok] - 1003-1091;DKHD
[尝试次数：5] - [剩余数量：1226] - [当前时间：11:07:13]

[ok] - 1003-1187;DKHD
[尝试次数：2] - [剩余数量：1225] - [当前时间：11:07:14]

[ok] - 1003-1141;DKHD
[尝试次数：4] - [剩余数量：1224] - [当前时间：11:07:15]

[ok] - 1003-1190;DKHD
[尝试次数：4] - [剩余数量：1223] - [当前时间：11:07:15]

[ok] - 1003-1192;DKHD
[尝试次数：1] - [剩余数量：1222] - [当前时间：11:07:16]

[ok] - 1003-1195;DKHD
[尝试次数：2] - [剩余数量：1221] - [当前时间：11:07:16]

[ok] - 273-10001;DKHD
[尝试次数：2] - [剩余数量：1220] - [当前时间：11:07:17]

[ok] - 1003-1193;DKHD
[尝试次数：3] - [剩余数量：1

[ok] - 210-1003;DKHD
[尝试次数：1] - [剩余数量：1105] - [当前时间：11:08:39]

[ok] - 210-1006;DKHD
[尝试次数：1] - [剩余数量：1104] - [当前时间：11:08:40]

[ok] - 273-10358;DKHD
[尝试次数：1] - [剩余数量：1103] - [当前时间：11:08:40]

[ok] - 1001-1059;DKHD
[尝试次数：1] - [剩余数量：1102] - [当前时间：11:08:40]

[ok] - 210-1007;DKHD
[尝试次数：1] - [剩余数量：1101] - [当前时间：11:08:40]

[ok] - 210-1012;DKHD
[尝试次数：1] - [剩余数量：1100] - [当前时间：11:08:40]

[ok] - 210-1010;DKHD
[尝试次数：1] - [剩余数量：1099] - [当前时间：11:08:41]

[ok] - 210-1013;DKHD
[尝试次数：1] - [剩余数量：1098] - [当前时间：11:08:41]

[ok] - 210-1009;DKHD
[尝试次数：1] - [剩余数量：1097] - [当前时间：11:08:41]

[ok] - 210-1016;DKHD
[尝试次数：1] - [剩余数量：1096] - [当前时间：11:08:41]

[ok] - 210-1019;DKHD
[尝试次数：1] - [剩余数量：1095] - [当前时间：11:08:42]

[ok] - 210-1020;DKHD
[尝试次数：1] - [剩余数量：1094] - [当前时间：11:08:43]

[ok] - 210-1021;DKHD
[尝试次数：1] - [剩余数量：1093] - [当前时间：11:08:44]

[ok] - 210-1017;DKHD
[尝试次数：1] - [剩余数量：1092] - [当前时间：11:08:44]

[ok] - 210-1027;DKHD
[尝试次数：1] - [剩余数量：1091] - [当前时间：11:08:45]

[ok] - 210-1023;DKHD
[尝试次数：1] - [剩余数量：1090] - [当前时间：1

[ok] - 211-2074;DKHD
[尝试次数：1] - [剩余数量：974] - [当前时间：11:09:53]

[ok] - 211-2080;DKHD
[尝试次数：1] - [剩余数量：973] - [当前时间：11:09:54]

[ok] - 211-2083;DKHD
[尝试次数：1] - [剩余数量：972] - [当前时间：11:09:56]

[ok] - 211-2085;DKHD
[尝试次数：1] - [剩余数量：971] - [当前时间：11:09:58]

[ok] - 211-2087;DKHD
[尝试次数：1] - [剩余数量：970] - [当前时间：11:09:59]

[ok] - 211-2089;DKHD
[尝试次数：1] - [剩余数量：969] - [当前时间：11:10:00]

[ok] - 211-2095;DKHD
[尝试次数：1] - [剩余数量：968] - [当前时间：11:10:01]

[ok] - 211-2102;DKHD
[尝试次数：1] - [剩余数量：967] - [当前时间：11:10:02]

[ok] - 211-2109;DKHD
[尝试次数：1] - [剩余数量：966] - [当前时间：11:10:02]

[ok] - 210-1035;DKHD
[尝试次数：2] - [剩余数量：965] - [当前时间：11:10:03]

[ok] - 211-2111;DKHD
[尝试次数：1] - [剩余数量：964] - [当前时间：11:10:04]

[ok] - 211-2130;DKHD
[尝试次数：1] - [剩余数量：963] - [当前时间：11:10:04]

[ok] - 210-1042;DKHD
[尝试次数：2] - [剩余数量：962] - [当前时间：11:10:05]

[ok] - 211-2142;DKHD
[尝试次数：1] - [剩余数量：961] - [当前时间：11:10:06]

[ok] - 211-2131;DKHD
[尝试次数：1] - [剩余数量：960] - [当前时间：11:10:06]

[ok] - 211-2155;DKHD
[尝试次数：1] - [剩余数量：959] - [当前时间：11:10:07]

[ok] - 2

...

[ok] - 250-241207;DKHD
[尝试次数：3] - [剩余数量：448] - [当前时间：11:13:57]

[ok] - 250-241208;DKHD
[尝试次数：3] - [剩余数量：447] - [当前时间：11:13:57]

[ok] - 250-241209;DKHD
[尝试次数：5] - [剩余数量：446] - [当前时间：11:13:58]

[ok] - 250-241210;DKHD
[尝试次数：2] - [剩余数量：445] - [当前时间：11:13:58]

[ok] - 250-241211;DKHD
[尝试次数：3] - [剩余数量：444] - [当前时间：11:13:59]

[ok] - 250-241212;DKHD
[尝试次数：1] - [剩余数量：443] - [当前时间：11:13:59]

[ok] - 250-241213;DKHD
[尝试次数：1] - [剩余数量：442] - [当前时间：11:13:59]

[ok] - 250-241205;DKHD
[尝试次数：7] - [剩余数量：441] - [当前时间：11:13:59]

[ok] - 250-241216;DKHD
[尝试次数：1] - [剩余数量：440] - [当前时间：11:14:00]

[ok] - 250-24121;DKHD
[尝试次数：4] - [剩余数量：439] - [当前时间：11:14:00]

[ok] - 250-241215;DKHD
[尝试次数：1] - [剩余数量：438] - [当前时间：11:14:00]

[ok] - 250-241217;DKHD
[尝试次数：2] - [剩余数量：437] - [当前时间：11:14:01]

[ok] - 250-241221;DKHD
[尝试次数：1] - [剩余数量：436] - [当前时间：11:14:01]

[ok] - 250-241222;DKHD
[尝试次数：1] - [剩余数量：435] - [当前时间：11:14:01]

[ok] - 250-241223;DKHD
[尝试次数：1] - [剩余数量：434] - [当前时间：11:14:02]

[ok] - 250-241214;DKHD
[尝试次数：3] - [剩余数量：4

[ok] - 250-24192;DKHD
[尝试次数：1] - [剩余数量：319] - [当前时间：11:15:01]

[ok] - 250-24194;DKHD
[尝试次数：1] - [剩余数量：318] - [当前时间：11:15:02]

[ok] - 250-24198;DKHD
[尝试次数：1] - [剩余数量：317] - [当前时间：11:15:03]

[ok] - 250-24200;DKHD
[尝试次数：1] - [剩余数量：316] - [当前时间：11:15:04]

[ok] - 250-24202;DKHD
[尝试次数：1] - [剩余数量：315] - [当前时间：11:15:06]

[ok] - 250-24203;DKHD
[尝试次数：1] - [剩余数量：314] - [当前时间：11:15:07]

[ok] - 250-24213;DKHD
[尝试次数：1] - [剩余数量：313] - [当前时间：11:15:08]

[ok] - 250-24220;DKHD
[尝试次数：1] - [剩余数量：312] - [当前时间：11:15:09]

[ok] - 250-24224;DKHD
[尝试次数：1] - [剩余数量：311] - [当前时间：11:15:10]

[ok] - 250-24225;DKHD
[尝试次数：1] - [剩余数量：310] - [当前时间：11:15:12]

[ok] - 250-24226;DKHD
[尝试次数：1] - [剩余数量：309] - [当前时间：11:15:12]

[ok] - 250-24228;DKHD
[尝试次数：1] - [剩余数量：308] - [当前时间：11:15:13]

[ok] - 250-24229;DKHD
[尝试次数：1] - [剩余数量：307] - [当前时间：11:15:14]

[ok] - 250-24230;DKHD
[尝试次数：1] - [剩余数量：306] - [当前时间：11:15:16]

[ok] - 250-24231;DKHD
[尝试次数：1] - [剩余数量：305] - [当前时间：11:15:17]

[ok] - 250-24232;DKHD
[尝试次数：1] - [剩余数量：304] - [当前时间：11:

[ok] - 250-24394;DKHD
[尝试次数：1] - [剩余数量：188] - [当前时间：11:16:39]

[ok] - 250-24392;DKHD
[尝试次数：1] - [剩余数量：187] - [当前时间：11:16:39]

[ok] - 250-24395;DKHD
[尝试次数：1] - [剩余数量：186] - [当前时间：11:16:40]

[ok] - 250-24397;DKHD
[尝试次数：1] - [剩余数量：185] - [当前时间：11:16:40]

[ok] - 250-24398;DKHD
[尝试次数：1] - [剩余数量：184] - [当前时间：11:16:40]

[ok] - 250-24399;DKHD
[尝试次数：1] - [剩余数量：183] - [当前时间：11:16:41]

[ok] - 250-24400;DKHD
[尝试次数：1] - [剩余数量：182] - [当前时间：11:16:42]

[ok] - 250-24402;DKHD
[尝试次数：1] - [剩余数量：181] - [当前时间：11:16:42]

[ok] - 250-24401;DKHD
[尝试次数：1] - [剩余数量：180] - [当前时间：11:16:43]

[ok] - 250-24403;DKHD
[尝试次数：1] - [剩余数量：179] - [当前时间：11:16:45]

[ok] - 250-24405;DKHD
[尝试次数：1] - [剩余数量：178] - [当前时间：11:16:45]

[ok] - 250-24406;DKHD
[尝试次数：1] - [剩余数量：177] - [当前时间：11:16:46]

[ok] - 250-24407;DKHD
[尝试次数：1] - [剩余数量：176] - [当前时间：11:16:47]

[ok] - 250-24408;DKHD
[尝试次数：1] - [剩余数量：175] - [当前时间：11:16:48]

[ok] - 250-24409;DKHD
[尝试次数：1] - [剩余数量：174] - [当前时间：11:16:49]

[ok] - 250-24410;DKHD
[尝试次数：1] - [剩余数量：173] - [当前时间：11:

[ok] - 250-24607;DKHD
[尝试次数：5] - [剩余数量：57] - [当前时间：11:17:39]

[ok] - 250-24618;DKHD
[尝试次数：4] - [剩余数量：56] - [当前时间：11:17:39]

[ok] - 250-24619;DKHD
[尝试次数：1] - [剩余数量：55] - [当前时间：11:17:40]

[ok] - 250-24623;DKHD
[尝试次数：1] - [剩余数量：54] - [当前时间：11:17:40]

[ok] - 250-24620;DKHD
[尝试次数：2] - [剩余数量：53] - [当前时间：11:17:40]

[ok] - 250-24625;DKHD
[尝试次数：4] - [剩余数量：52] - [当前时间：11:17:42]

[ok] - 250-24613;DKHD
[尝试次数：12] - [剩余数量：51] - [当前时间：11:17:42]

[ok] - 250-24621;DKHD
[尝试次数：9] - [剩余数量：50] - [当前时间：11:17:43]

[ok] - 250-24628;DKHD
[尝试次数：3] - [剩余数量：49] - [当前时间：11:17:43]

[ok] - 250-24626;DKHD
[尝试次数：4] - [剩余数量：48] - [当前时间：11:17:43]

[ok] - 250-24629;DKHD
[尝试次数：1] - [剩余数量：47] - [当前时间：11:17:44]

[ok] - 250-24630;DKHD
[尝试次数：1] - [剩余数量：46] - [当前时间：11:17:44]

[ok] - 250-24632;DKHD
[尝试次数：1] - [剩余数量：45] - [当前时间：11:17:45]

[ok] - 250-24634;DKHD
[尝试次数：1] - [剩余数量：44] - [当前时间：11:17:45]

[ok] - 250-24631;DKHD
[尝试次数：3] - [剩余数量：43] - [当前时间：11:17:45]

[ok] - 250-24637;DKHD
[尝试次数：2] - [剩余数量：42] - [当前时间：11:17:46]

[ok] - 

KeyError: '序号'